### UNCOMMENT AND RUN CELL TO INSTALL DEPENDANCIES

In [9]:
### UNCOMMENT AND RUN CELL TO INSTALL DEPENDANCIES

import pip
# pip.main(['install', 'python-osc'])
# pip.main(['install', 'numpy'])
# pip.main(['install', 'mido'])
# pip.main(['install', 'python-rtmidi'])

In [ ]:
import argparse
import math, time
import numpy as np

from pythonosc import dispatcher
from pythonosc import osc_server
import asyncio

### Discover and establish virtual MIDI port

To create a new virtual MIDI port of your machine, see the following instructions for Windows and OSX:

https://help.ableton.com/hc/en-us/articles/209774225-Using-virtual-MIDI-buses

In [2]:
import time
import mido
import rtmidi

midiout = rtmidi.MidiOut()
available_ports = midiout.get_ports()
print("available ports: %s" % available_ports)
if available_ports:
    port_num = 0
    midiout.open_port(0)
    print("connected to: '%s'" % available_ports[port_num])
else:
    midiout.open_virtual_port("NAME OF VIRTUAL MIDI PORT")

available ports: ['To max 2 2m1', 'To max 2 2m2']
connected to: 'To max 2 2m1'


### Parse OSC data and convert to MIDI Control Change messages

In [3]:
prev_ctrl_vals = None
def assign_midi_data(msg_addr, *values):
    global prev_ctrl_vals
    list(values)
    ##Process
    
    if prev_ctrl_vals is None:
        prev_ctrl_vals = np.zeros(len(values))
    new_ctrl_values = np.array(values)
    for i, val in enumerate(new_ctrl_values):
        if(val != prev_ctrl_vals[i]):
            val = int(round(val*127.0))
            control_change = mido.Message('control_change', channel=0, control=i, value=val).bytes()
            midiout.send_message(control_change)
    prev_ctrl_vals = new_ctrl_values

### Configure OSC listener

In [4]:
## Change these according to the server output
osc_recv_addr = "/wek/outputs"
osc_recv_port = 12000

main_loop = asyncio.get_event_loop()

dispatcher = dispatcher.Dispatcher()
dispatcher.map(osc_recv_addr, assign_midi_data)

server = osc_server.AsyncIOOSCUDPServer(
    ('127.0.0.1', osc_recv_port), dispatcher, main_loop)
print("Serving on {}".format(server._server_address))
server.serve()
main_loop.run_forever()

Serving on ('127.0.0.1', 12000)


KeyboardInterrupt: 

In [ ]:
# del midiout